In [1]:
import numpy as np
import pandas as pd
from my_stuff import BinanceUSDMTestKeys, ApexTestKeys, MufexTestKeys, BybitTestKeys
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from quantfreedom.exchanges.binance_exchange.binance_usdm import BinanceUSDM
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit
from quantfreedom.enums import PositionModeType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.exchanges.apex_exchange.apex import Apex
from quantfreedom.indicators.tv_indicators import *
np.set_printoptions(formatter={"float_kind": "{:0.1f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(exchange="binance_usdm", symbol="BTCUSDT", timeframe="30m", candles_to_dl=20)
open = candles[:, 1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4], 1)
candles

array([[1700123400000.0, 37318.9, 37427.6, 37253.6, 37327.9],
       [1700125200000.0, 37328.0, 37473.0, 37211.0, 37454.0],
       [1700127000000.0, 37454.0, 37477.6, 37273.7, 37380.2],
       [1700128800000.0, 37380.2, 37411.0, 37362.7, 37406.1],
       [1700130600000.0, 37406.0, 37411.1, 37315.9, 37317.5],
       [1700132400000.0, 37317.5, 37574.9, 37294.9, 37549.4],
       [1700134200000.0, 37549.4, 37563.9, 37150.0, 37272.1],
       [1700136000000.0, 37272.0, 37352.5, 37076.4, 37095.0],
       [1700137800000.0, 37095.0, 37290.5, 37068.2, 37153.4],
       [1700139600000.0, 37153.3, 37188.3, 36854.8, 36929.1],
       [1700141400000.0, 36929.2, 36977.3, 36701.0, 36797.9],
       [1700143200000.0, 36797.9, 36880.0, 36505.0, 36560.0],
       [1700145000000.0, 36559.9, 36839.4, 36451.0, 36725.5],
       [1700146800000.0, 36725.5, 36926.4, 36690.7, 36876.2],
       [1700148600000.0, 36876.2, 36932.8, 36251.0, 36423.1],
       [1700150400000.0, 36423.3, 36658.0, 36387.2, 36576.3],
       [

In [3]:
candles_to_df(candles=candles)

,timestamp,open,high,low,close
datetime,,,,,
2023-11-16 08:30:00,1700123400000,37318.9,37427.6,37253.6,37327.9
2023-11-16 09:00:00,1700125200000,37328.0,37473.0,37211.0,37454.0
2023-11-16 09:30:00,1700127000000,37454.0,37477.6,37273.7,37380.2
2023-11-16 10:00:00,1700128800000,37380.2,37411.0,37362.7,37406.1
2023-11-16 10:30:00,1700130600000,37406.0,37411.1,37315.9,37317.5
2023-11-16 11:00:00,1700132400000,37317.5,37574.9,37294.9,37549.4
2023-11-16 11:30:00,1700134200000,37549.4,37563.9,37150.0,37272.1
2023-11-16 12:00:00,1700136000000,37272.0,37352.5,37076.4,37095.0
2023-11-16 12:30:00,1700137800000,37095.0,37290.5,37068.2,37153.4


In [4]:
mufex_test = Mufex(
    api_key=MufexTestKeys.api_key,
    secret_key=MufexTestKeys.secret_key,
    use_test_net=True,
)
mufex_test.create_order(
    symbol="BTCUSDT",
    position_mode=PositionModeType.BuySide,
    buy_sell="buy",
    order_type="market",
    asset_size=0.001,
    time_in_force="GoodTillCancel",
)

'46c21c82-a1be-4d02-a8e0-179cd2c907f7'

In [5]:
apex_test = Apex(
    api_key=ApexTestKeys.api_key,
    secret_key=ApexTestKeys.secret_key,
    passphrase=ApexTestKeys.passphrase,
    stark_key_private=ApexTestKeys.stark_key_private,
    stark_key_public=ApexTestKeys.stark_key_public,
    stark_key_y=ApexTestKeys.stark_key_y,
    use_test_net=True
)
apex_test.create_entry_market_order(
    symbol="BTC-USDC",
    buy_sell="buy",
    asset_size=0.005,
)

'513059596743475308'

In [6]:
binance_test = BinanceUSDM(
    api_key=BinanceUSDMTestKeys.api_key,
    secret_key=BinanceUSDMTestKeys.secret_key,
    use_test_net=True,
)
binance_test.create_order(
    symbol="BTCUSDT",
    side="BUY",
    type="MARKET",
    quantity=0.005,
    timeInForce=None,
)

{'orderId': 3527361067,
 'symbol': 'BTCUSDT',
 'status': 'NEW',
 'clientOrderId': '7XcqZoC1XLECDAZxn1O4iu',
 'price': '0.00',
 'avgPrice': '0.00',
 'origQty': '0.005',
 'executedQty': '0.000',
 'cumQty': '0.000',
 'cumQuote': '0.00000',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'reduceOnly': False,
 'closePosition': False,
 'side': 'BUY',
 'positionSide': 'BOTH',
 'stopPrice': '0.00',
 'workingType': 'CONTRACT_PRICE',
 'priceProtect': False,
 'origType': 'MARKET',
 'priceMatch': 'NONE',
 'selfTradePreventionMode': 'NONE',
 'goodTillDate': 0,
 'updateTime': 1700159740975}

In [7]:
bybit_test = Bybit(
    use_test_net=True,
    api_key=BybitTestKeys.api_key,
    secret_key=BybitTestKeys.secret_key,
)
bybit_test.create_order(
    symbol="BTCUSDT",
    buy_sell="buy",
    position_mode=1,
    order_type="market",
    asset_size=0.01,
)

'ca7a6992-907f-4e05-8db5-71267328dabd'